In [16]:
import pandas as pd
import unicodedata

df = pd.read_csv('dados_tratados.csv')
df.head()


,uf,mes_referencia,idade,sexo,escolaridade,teve_febre,teve_tosse,teve_dor_garganta,teve_dificuldade_respirar,teve_dor_peito,perda_olfato_paladar,procurou_saude,resultado_exame,medida_isolamento,trabalhou_semana_passada,afastado_trabalho,tipo_trabalho,faixa_rendimento,trabalho_remoto,auxilio_emergencial
0,Rondônia,9,36,Homem,Médio completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,Outro técnico ou profissional de nível médio,801 - 1.600,Não aplicável,Sim
1,Rondônia,9,30,Mulher,Superior completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,"Técnico, profissional da saúde de nível médio",801 - 1.600,Não aplicável,Sim
2,Rondônia,9,13,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
3,Rondônia,9,11,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
4,Rondônia,9,57,Mulher,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Não,Não,não aplicável,Não aplicável,Não aplicável,Não


In [17]:
sym_long_freq = df.melt(
    value_vars=sintoma_cols,
    var_name='sintoma',
    value_name='resposta',
)
sym_long_freq['resposta_norm'] = sym_long_freq['resposta'].map(lambda s: to_ascii(str(s)))

freq_counts = (
    sym_long_freq.groupby(['resposta_norm', 'sintoma'])
    .size()
    .reset_index(name='contagem')
)
freq_counts['pct'] = (
    freq_counts.groupby('sintoma')['contagem']
    .transform(lambda s: (s / s.sum()) * 100)
    .round(6)
)

frequencia_sintomas = freq_counts.pivot_table(
    index='resposta_norm',
    columns='sintoma',
    values='pct',
    fill_value=0,
)

resposta_order = ['nao', 'sim', 'ignorado', 'nao_sabe']
resposta_labels = {
    'nao': 'Nao',
    'sim': 'Sim',
    'ignorado': 'Ignorado',
    'nao_sabe': 'Nao_sabe',
}
sintoma_labels = {
    'teve_dificuldade_respirar': 'Dificuldade de respirar',
    'teve_febre': 'Teve Febre',
    'teve_tosse': 'Teve Tosse',
    'teve_dor_garganta': 'Teve dor de garganta',
    'teve_dor_peito': 'Teve dor no peito',
    'perda_olfato_paladar': 'Perda de olfato e paladar',
}

frequencia_sintomas = frequencia_sintomas.reindex(columns=sintoma_cols)
frequencia_sintomas = frequencia_sintomas.reindex(resposta_order)
frequencia_sintomas = frequencia_sintomas.rename(index=resposta_labels)
frequencia_sintomas = frequencia_sintomas.rename(columns=sintoma_labels)

frequencia_sintomas


sintoma,Teve Febre,Teve Tosse,Teve dor de garganta,Dificuldade de respirar,Teve dor no peito,Perda de olfato e paladar
resposta_norm,,,,,,
Nao,NaN,NaN,NaN,NaN,NaN,NaN
Sim,0.796286,1.430423,1.094506,0.469922,0.341081,0.421381
Ignorado,0.359930,0.365610,0.364577,0.365094,0.366385,0.459336
Nao_sabe,0.055513,0.056546,0.063001,0.059386,0.065324,0.065841


In [18]:
# padroniza textos
obj_cols = df.select_dtypes(include='object').columns
df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())

sintoma_cols = [
    'teve_febre',
    'teve_tosse',
    'teve_dor_garganta',
    'teve_dificuldade_respirar',
    'teve_dor_peito',
    'perda_olfato_paladar',
]

bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, 120]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['faixa_etaria'] = pd.cut(df['idade'], bins=bins, labels=labels, include_lowest=True)

df['tem_sintoma_chave'] = df[sintoma_cols].eq('Sim').any(axis=1)

def to_ascii(text):
    return (
        unicodedata.normalize('NFKD', text)
        .encode('ascii', 'ignore')
        .decode('ascii')
        .lower()
        .replace(' ', '_')
    )


In [19]:
sym_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=sintoma_cols,
    var_name='sintoma',
    value_name='resposta',
)
sym_long['resposta_norm'] = sym_long['resposta'].map(lambda s: to_ascii(str(s)))
sym_long['resposta_norm'] = sym_long['resposta_norm'].replace({'nao_sabe': 'ignorado'})

sym_counts = (
    sym_long.groupby(['mes_referencia', 'sintoma', 'resposta_norm'])
    .size()
    .reset_index(name='contagem')
)
sym_counts['pct'] = (
    sym_counts.groupby(['mes_referencia', 'sintoma'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)

gold_sintomas_mes = sym_counts.pivot_table(
    index=['mes_referencia', 'sintoma'],
    columns='resposta_norm',
    values=['contagem', 'pct'],
    fill_value=0,
)
gold_sintomas_mes.columns = [
    f"{metric}_{resp}" for metric, resp in gold_sintomas_mes.columns
]
gold_sintomas_mes = gold_sintomas_mes.reset_index()
gold_sintomas_mes.head()


,mes_referencia,sintoma,contagem_ignorado,contagem_nao,contagem_sim,pct_ignorado,pct_nao,pct_sim
0,9,perda_olfato_paladar,2034.0,383632.0,1632.0,0.0053,0.9905,0.0042
1,9,teve_dificuldade_respirar,1644.0,383834.0,1820.0,0.0042,0.9911,0.0047
2,9,teve_dor_garganta,1656.0,381403.0,4239.0,0.0043,0.9848,0.0109
3,9,teve_dor_peito,1672.0,384305.0,1321.0,0.0043,0.9923,0.0034
4,9,teve_febre,1609.0,382605.0,3084.0,0.0042,0.9879,0.0080


In [20]:
gold_sintomas_perfil = (
    df.groupby(['mes_referencia', 'faixa_etaria', 'sexo', 'uf'])
    .agg(total=('tem_sintoma_chave', 'size'), com_sintoma=('tem_sintoma_chave', 'sum'))
    .reset_index()
)
gold_sintomas_perfil['pct_com_sintoma_chave'] = (
    (gold_sintomas_perfil['com_sintoma'] / gold_sintomas_perfil['total']).round(4)
)
gold_sintomas_perfil.head()


C:\Users\00157NLUC-BrenoR\AppData\Local\Temp\ipykernel_12840\1550526282.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['mes_referencia', 'faixa_etaria', 'sexo', 'uf'])


,mes_referencia,faixa_etaria,sexo,uf,total,com_sintoma,pct_com_sintoma_chave
0,9,0-9,Homem,Acre,461,5,0.0108
1,9,0-9,Homem,Alagoas,802,17,0.0212
2,9,0-9,Homem,Amapá,199,5,0.0251
3,9,0-9,Homem,Amazonas,816,15,0.0184
4,9,0-9,Homem,Bahia,1011,20,0.0198


In [21]:
economic_cols = [
    'trabalhou_semana_passada',
    'afastado_trabalho',
    'tipo_trabalho',
    'faixa_rendimento',
    'trabalho_remoto',
    'auxilio_emergencial',
]

econ_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=economic_cols,
    var_name='indicador_economico',
    value_name='categoria',
)

gold_economia_mes = (
    econ_long.groupby(['mes_referencia', 'indicador_economico', 'categoria'])
    .size()
    .reset_index(name='contagem')
)
gold_economia_mes['pct'] = (
    gold_economia_mes.groupby(['mes_referencia', 'indicador_economico'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)
gold_economia_mes.head()


,mes_referencia,indicador_economico,categoria,contagem,pct
0,9,afastado_trabalho,Não,163004,0.4209
1,9,afastado_trabalho,Não aplicável,208466,0.5383
2,9,afastado_trabalho,Sim,15828,0.0409
3,9,auxilio_emergencial,Não,187429,0.4839
4,9,auxilio_emergencial,Sim,199869,0.5161


In [22]:
behavior_cols = ['medida_isolamento', 'procurou_saude']

behavior_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=behavior_cols,
    var_name='indicador_comportamento',
    value_name='categoria',
)

gold_comportamento_mes = (
    behavior_long.groupby(['mes_referencia', 'indicador_comportamento', 'categoria'])
    .size()
    .reset_index(name='contagem')
)
gold_comportamento_mes['pct'] = (
    gold_comportamento_mes.groupby(['mes_referencia', 'indicador_comportamento'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)

gold_sintomas_mes.to_csv('gold_sintomas_mes.csv', index=False)
gold_sintomas_perfil.to_csv('gold_sintomas_perfil.csv', index=False)
gold_economia_mes.to_csv('gold_economia_mes.csv', index=False)
gold_comportamento_mes.to_csv('gold_comportamento_mes.csv', index=False)

gold_comportamento_mes.head()


,mes_referencia,indicador_comportamento,categoria,contagem,pct
0,9,medida_isolamento,Ficou em casa e só saiu em caso de necessidade...,161444,0.4168
1,9,medida_isolamento,Ficou rigorosamente em casa,63863,0.1649
2,9,medida_isolamento,Ignorado,1926,0.0050
3,9,medida_isolamento,"Não fez restrição, levou vida normal como ante...",11191,0.0289
4,9,medida_isolamento,"Reduziu o contato com as pessoas, mas continuo...",148874,0.3844
